# Chapter 2 - JSONS & Dictionaries: Using Pydantic

Working with small and simple Dictionaries is easy, but as you start scaling and working with larger, more complex objects, the dynamic typing of Python makes this more difficult.

Fortunately, [**Pydantic**](https://pydantic-docs.helpmanual.io/) is a powerful library that solves this!

### Deck of Cards API

In this notebook, we will use the [Deck of Cards API](https://deckofcardsapi.com/) which is a free, open source API for playing with face cards!

❤️ ♠️ ♦️ ♣️
---
---

### Pydantic

You will need to install `pydantic` in your project.

>💡 In Google Colab, `pydantic` is **NOT already** installed!

```python
pip install pydantic

# or your preferred package manager

pipenv install pydantic

# ---or---

poetry add pydantic
```

In [ ]:
""" Install pydantic (only if needed)

* Don't run this cell if you already have pydantic installed in your project
* Run this cell if you are running this notebook in Google Colaboratory
"""
!pip install pydantic

In [1]:
""" Continue from where we left off in the previous chapter """
import requests

URL = 'https://deckofcardsapi.com/api/deck/new/shuffle/?deck_count=1'
response = requests.get(URL)
print(response.json())

{'success': True, 'deck_id': 'f3p3jssmam15', 'remaining': 52, 'shuffled': True}


In [2]:
""" Represent this JSON as a Pydantic Model """
from pydantic import BaseModel

class NewDeckModel(BaseModel):
    deck_id   : str
    remaining : int
    success   : bool
    shuffled  : bool

In [3]:
""" Unpack the Response JSON into the Model """
new_deck = NewDeckModel(**response.json())

# no errors means that the JSON adheres to the Model Schema!
# change deck_id to `int` in the Model and see what happens
print(new_deck)

deck_id='f3p3jssmam15' remaining=52 success=True shuffled=True


In [4]:
""" We also get intellisense & autocomplete! """
print(new_deck.deck_id)   # instead of new_deck['deck_id'] with no intellisense
print(new_deck.remaining)
print(new_deck.success)
print(new_deck.shuffled)

f3p3jssmam15
52
True
True


## Models that share fields

As you deal with a lot of these Models and Schemas, there will be times when you see **shared fields**.

For example, when you draw cards from this API, the Response also includes:

* `deck_id`
* `remaining`
* `success`

>💡 This is very close to the `NewDeckModel` except for the `shuffled` field.

### The challenge
---
Create a `DeckModel` class that the `NewDeckModel` can use _without_ duplicating fields.

>💭 Eventually, we will create a `DrawCardsModel` class that can _also_ use it.

In [5]:
from pydantic import BaseModel

class DeckModel(BaseModel):
    deck_id   : str
    remaining : int
    success   : bool


class NewDeckModel(DeckModel):
    shuffled: bool


new_deck = NewDeckModel(**response.json())
print(new_deck)

deck_id='f3p3jssmam15' remaining=52 success=True shuffled=True


## Use data from Response to make another Request

It is very common to use data from one Request or Response to make another Request.

For example, your app might have this flow:

1. Create a User with the `/user` endpoint which returns a `UserModel` object
2. Then authenticate the user given their `user.id` with the `/auth` endpoint

---
👀 Look at the URL for the Draw Cards endpoint. It requires:

* `deck_id` of the deck to draw from
* `count` for the number of cards to draw

```
https://deckofcardsapi.com/api/deck/<<deck_id>>/draw/?count=2
```

💪🏽 You probably figured it out, but we already have the `deck_id`!

In [6]:
""" Use our new_deck to draw 2 cards from it """
DRAW_CARDS_URL = f'https://deckofcardsapi.com/api/deck/{new_deck.deck_id}/draw/?count=2'

response = requests.get(DRAW_CARDS_URL)
print(response.json())

{'success': True, 'deck_id': 'f3p3jssmam15', 'cards': [{'code': 'QD', 'image': 'https://deckofcardsapi.com/static/img/QD.png', 'images': {'svg': 'https://deckofcardsapi.com/static/img/QD.svg', 'png': 'https://deckofcardsapi.com/static/img/QD.png'}, 'value': 'QUEEN', 'suit': 'DIAMONDS'}, {'code': '0H', 'image': 'https://deckofcardsapi.com/static/img/0H.png', 'images': {'svg': 'https://deckofcardsapi.com/static/img/0H.svg', 'png': 'https://deckofcardsapi.com/static/img/0H.png'}, 'value': '10', 'suit': 'HEARTS'}], 'remaining': 50}


## Your Challenge

Create the `DrawCardsModel` class and unpack the `response.json()` into it successfully.

* Your model **must** use the `DeckModel`
* Unpacking `response.json()` should raise no errors
* Assert that 2 cards are drawn

>💡 **HINT:** The `cards` field is a list of Card objects, so make a `Card` or `CardModel` class to represent it.

```python
from typing import List

class Card(BaseModel):
    ...

class Example(BaseModel):
    cards: List[Card]
```